In [ ]:
# author: Jana Lasser

In [1]:
import pandas as pd
import numpy as np
from os.path import join
import os

In [2]:
dst = "../../data/users/"

In [3]:
if not os.path.exists(join(dst, "raw")): os.mkdir(join(dst, "raw"))
if not os.path.exists(join(dst, "clean")): os.mkdir(join(dst, "clean"))

# Download data

In [ ]:
# 114th congress (only senators!)
url = "https://www.socialseer.com/resources/us-senator-twitter-accounts-114th-congress-historical/"
df = pd.read_html(url)[0]
df.columns = df.loc[0]
df = df.drop(0)
df["congress"] = 114
df.to_csv(join(dst, "raw", "114_congress.csv"), index=False)

In [ ]:
# 115th congress (only senators!)
url = "https://www.socialseer.com/resources/us-senator-twitter-accounts/"
df = pd.read_html(url)[0]
df.columns = df.loc[0]
df = df.drop(0)
df["congress"] = 115
df.to_csv(join(dst, "raw", "115_congress.csv"), index=False)

In [7]:
# 116 congress (senate and house) manually downloaded from 
# https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/MBOJNS
df_house = pd.read_csv(join(dst, "raw", "congress116-house-accounts.csv"))
df_senate = pd.read_csv(join(dst, "raw", "congress116-senate-accounts.csv"))
df_house["chamber"] = "house"
df_senate["chamber"] = "senate"
df = pd.concat([df_house, df_senate])
df["congress"] = 116
df.to_csv(join(dst, "raw", "116_congress.csv"), index=False)

In [4]:
# 117th congress (senate and house, downloaded on 2022-03-14)
url = "https://triagecancer.org/congressional-social-media"
df = pd.read_html(url)[0]
df["congress"] = 117
df.to_csv(join(dst, "raw", "117_congress.csv"), index=False)

In [43]:
# 118th congress (house)
# manually downloaded from https://pressgallery.house.gov/member-data/members-official-twitter-handles
df = pd.read_csv(join(dst, "raw", "congress118-house-accounts.tsv"), delimiter="\t")
df["congress"] = 118
df.to_csv(join(dst, "raw", "118_congress.csv"), index=False)

# Clean and merge data

In [5]:
df_114 = pd.read_csv(join(dst, "raw", "114_congress.csv"), 
                     usecols = ["Official Senator Twitter ID",
                                "Senate Staff Twitter Id",
                                "Campaign/Other ID", 
                                "congress"])
df_115 = pd.read_csv(join(dst, "raw", "115_congress.csv"),
                    usecols = ["Official Twitter", 
                               "Staff Twitter",
                               "Campaign Twitter",
                               "congress"])
df_116 = pd.read_csv(join(dst, "raw", "116_congress.csv"),
                    usecols=["Token", "congress"])
df_117 = pd.read_csv(join(dst, "raw", "117_congress.csv"),
                    usecols=["Twitter", "congress"])
df_118 = pd.read_csv(join(dst, "raw", "118_congress.csv"),
                    usecols=["Twitter Handle", "congress"])

In [6]:
df_114 = df_114.rename(columns={"Official Senator Twitter ID":"official",
                                "Senate Staff Twitter Id":"staff",
                                "Campaign/Other ID":"campaign"})
df_114 = pd.melt(df_114, value_vars=["official", "staff", "campaign"], 
        id_vars=["congress"], var_name="type", value_name="handle")

df_115 = df_115.rename(columns={"Official Twitter":"official",
                                "Staff Twitter":"staff",
                                "Campaign Twitter":"campaign"})
df_115 = pd.melt(df_115, value_vars=["official", "staff", "campaign"], 
        id_vars=["congress"], var_name="type", value_name="handle")

df_116 = df_116.rename(columns={"Token":"handle"})
df_116["type"] = "official"
df_117 = df_117.rename(columns={"Twitter":"handle"})
df_117["type"] = "official"
df_118 = df_118.rename(columns={"Twitter Handle":"handle"})
df_118["type"] = "official"

In [7]:
df_114 = pd.concat([
    df_114, 
    pd.DataFrame([{"congress":114, "type":"campaign", "handle":"@randpaul16"}])
])

df_114 = pd.concat([
    df_114, 
    pd.DataFrame([{"congress":114, "type":"campaign", "handle":"@DrRandPaul"}])
])
df_114 = df_114.drop(236).reset_index(drop=True)

In [8]:
print(f"{len(df_114.dropna())} handles found for 114th congress")
print(f"{len(df_115.dropna())} handles found for 115th congress")
print(f"{len(df_116.dropna())} handles found for 116th congress")
print(f"{len(df_117.dropna())} handles found for 117th congress")
print(f"{len(df_118.dropna())} handles found for 118th congress")

198 handles found for 114th congress
202 handles found for 115th congress
944 handles found for 116th congress
538 handles found for 117th congress
432 handles found for 118th congress


In [9]:
df = pd.concat([df_114, df_115, df_116, df_117, df_118])
df = df.dropna()
df = df.reset_index(drop=True)
df["handle"] = df["handle"].apply(lambda x: x.replace("@", ""))
df["handle"] = df["handle"].apply(lambda x: x.split(" ")[0])

In [10]:
print(f"found a total of {len(df)} handles")
print(f"found a total of {len(df.handle.unique())} unique handles")

found a total of 2314 handles
found a total of 1278 unique handles


# Download current account information from the Twitter API

In [89]:
df.to_csv(join(dst, "clean", "congress-member-twitter-handles_114-118.csv"), index=False)
np.savetxt(join(dst, "clean", "congress-member-twitter-handles_114-118.txt"), 
           df["handle"].drop_duplicates().values, fmt="%s")

In [90]:
# get twitter account information
! twarc2 users --usernames ../../data/users/clean/congress-member-twitter-handles_114-118.txt ../../data/users/US_politician_twitter_accounts.jsonl

100%|████████| Processed 1278/1278 lines of input file [00:08<00:00, 159.40it/s]


In [91]:
# transform account information from JSON to csv
! twarc2 csv --input-data-type users ../../data/users/US_politician_twitter_accounts.jsonl ../../data/users/US_politician_twitter_accounts.csv

100%|██████████████| Processed 2.14M/2.14M of input file [00:00<00:00, 17.0MB/s]

ℹ️
Parsed 1169 users objects from 13 lines in the input file.
17 were duplicates. Wrote 1152 rows and output 27 columns in the CSV.



# Clean user data

In [3]:
src = "../../data/users"
fname = "US_politician_twitter_accounts.csv"
users = pd.read_csv(join(src, fname), dtype={"id":str})
users.head(2)

,id,created_at,username,name,description,entities.description.cashtags,entities.description.hashtags,entities.description.mentions,entities.description.urls,entities.url.urls,...,public_metrics.tweet_count,url,verified,verified_type,withheld.scope,withheld.copyright,withheld.country_codes,__twarc.retrieved_at,__twarc.url,__twarc.version
0,18061669,2008-12-11T21:55:19.000Z,lisamurkowski,Sen. Lisa Murkowski,Official Twitter account for U.S. Senator Lisa...,NaN,NaN,NaN,NaN,"[""http://murkowski.senate.gov""]",...,13225,http://murkowski.senate.gov,True,NaN,NaN,NaN,NaN,2023-02-12T09:10:54+00:00,https://api.twitter.com/2/users/by?tweet.field...,2.13.0
1,19394188,2009-01-23T13:42:31.000Z,SenJohnMcCain,John McCain,"Memorial account for U.S. Senator John McCain,...",NaN,NaN,"[""@McCainInstitute""]",NaN,NaN,...,14397,NaN,True,NaN,NaN,NaN,NaN,2023-02-12T09:10:54+00:00,https://api.twitter.com/2/users/by?tweet.field...,2.13.0


In [4]:
df = pd.read_csv(join(src, "clean", "congress-member-twitter-handles_114-118.csv"))
diff = list(set(df["handle"]).difference(set(users["username"])))
print(f"{len(diff)} user profiles could not be retrieved")

136 user profiles could not be retrieved


In [5]:
users = users.rename(columns={
    "id":"author_id",
    "public_metrics.following_count":"following_count",
    "public_metrics.followers_count":"followers_count",
    "public_metrics.tweet_count":"tweet_count"
})

In [6]:
src = "../../data/utilities"
party_affiliation = pd.read_csv(join(src, "party_affiliations_complete.csv"),
                dtype={"author_id":str, "name":str, "handle":str, "party":str})
party_affiliation = party_affiliation.rename(columns={"handle":"username"})

In [7]:
# remove accounts after manual checking
# @RepTomCotton: fake account
# @repcorygardner: fake account
# @RepGaryPeters: fake account
# @senatorsanders: unverified (old?) account
# @SenBookerOfc: fake account
# @coons4delaware: unverified old account
# @DebFischer2012: not a politician account
# @Menendez4NJ: not a politician account
# @ToomeyForSenate: not a politician account
# @friendsofbernie: not a politician account
# @McCaskill2018: not a politician account
# @DebFischerNE: not a politician account
# @AnthonyGonzalez: not a politician account
# @BuddForCongress: not a politician account
# @RichHudson: not a politician account
# @senatorvitter not the official account
# @RepRobBishop not the official account
# @RepTomReed: not a politician account

N = len(users)
users = pd.merge(users, party_affiliation[["username", "party"]],
                 how="left", left_on="username", right_on="username")
users = users.dropna(subset=["party"])
print(f"dropped {N - len(users)} accounts")
users = users.rename(columns={"username":"handle"})

dropped 14 accounts


In [61]:
dst = "../../data/users"
cols = ["author_id", "handle", "name", "description",
        "followers_count", "following_count", "tweet_count",
        "created_at", "party"]
fname = "US_politician_twitter_accounts_clean.csv"
users[cols].to_csv(join(dst, fname), index=False)